In [1]:
#########################################
#           Python Version              #
# 3.7.7 (default, May  6 2020, 04:59:01)# 
# [Clang 4.0.1 (tags/RELEASE_401/final)]#
#########################################

In [2]:
from docx import Document  
from docx.oxml.ns import qn  
from docx.shared import Pt, RGBColor

In [3]:
def formatQuoteAt(lst):
    k = []
    for j in range(len(lst)):
        if "CQ:quote" in lst[j] and j != 1:
            lst[j] = lst[j].split("[")[0]
            k.append(lst[j])
        elif "CQ:quote" in lst[j]:
            pass
        elif "CQ:at" in lst[j]:
            at_start = lst[j].find("@")
            lst[j] = lst[j][at_start:]
            at_end = lst[j].find("&")
            if (at_end != -1):
                lst[j] = lst[j][:at_end] + "\t"
            else:
                lst[j] = lst[j] + "\t"
            k.append(lst[j])
        elif "CQ:At" in lst[j]:
            at_start = lst[j].find("[")
            lst[j] = lst[j][:at_start]
            k.append(lst[j])
        else:
            k.append(lst[j])
    lst = [k[0]]
    for p in range(1,len(k)):
        lst = appendNonRepeat(lst, k[p])
    return lst

In [4]:
def appendNonRepeat(clean, strip):
    if (strip not in clean[-1]):
        clean.append(strip)
    return clean

In [5]:
def extractAlias(text):
    aliasLst = []
    cardNames = text.split("]:")
    for cardName in cardNames[:-1]:
        cardName = cardName.split('[')[-1]
        aliasLst.append(cardName)
    atNames = text.split("&#93;.display]")
    for atName in atNames[:-1]:
        atName = atName.split('@')[-1]
        aliasLst.append(atName)
    aliasLst = list(set(aliasLst))
    alias = {}
    for key in aliasLst:
        alias[key] = ''
    return alias

In [6]:
def enTransToC(string):
    enPun = u',~!?()"\''
    cPun = u'，～！？（）“‘'
    table= {ord(f):ord(t) for f,t in zip(enPun,cPun)}
    return string.translate(table)

In [7]:
def replaceAlias(text, alias):
    for key in alias.keys():
        text = text.replace(key, alias[key])
    return text

In [8]:
def removeCQBot(textLst):
    textLst = [i for i in textLst if 
               ("msg brief" not in i) and 
               ("CQ:image" not in i) and 
               ("切换到人物卡" not in i) and 
               ("CQ:service" not in i) and 
               ("此版本更新" not in i) and
               ("对塔骰进行投喂" not in i) and
               ("已为kp自定义为角色" not in i) and
               ("加入小队" not in i)
              ]
    return textLst

In [9]:
# read file and draft alias
def stepOne(path, encoding):
    # read file
    with open(path, encoding=encoding) as f:
        text = f.read()
    # extract dictionary
    print(extractAlias(text))
    return text

In [10]:
# split and clean by line 
def stepTwo(text, alias):
    # replace alias
    text = replaceAlias(text, alias)
    
    # split to list
    textLst = text.split("\n")
    # remove image and reference
    textLst = removeCQBot(textLst)
    textLst = [i.strip() for i in textLst]
    textLst = [i for i in textLst if i]

    # clean by line    
    clean = ['']
    strip = ''
    for i in textLst:
        # detect start of a message
        if (i[0] == '<'):
            clean = appendNonRepeat(clean, strip)
            i = i.split("]")
            # detect bot message(with bot name and player name)
            if (len(i)==3 and "bot" in i[0]):
                i[1] = i[1] + "]"
                i[-1] = i[-1] + " "
            # detect quote and at
            elif (len(i)>2):
                i = formatQuoteAt(i)
                if ':\t' != i[1][:2]:
                    i[0] = i[0] +':\t'
            i[0] = i[0].split("[")[1]
            strip = "".join(i)
        else:
            i = i.split("]")
            i = formatQuoteAt(i)
            i = "".join(i)
            strip = strip + i
    clean = appendNonRepeat(clean, strip)
    clean = "\n".join(clean[1:])
    
    # fix punctuations
    clean = enTransToC(clean)
    
    return clean

In [11]:
# encoding and write file
def stepThree(clean, encoding, path):
    clean.encode(encoding)
    with open(path, "w", encoding=encoding) as f:
        f.write(clean)

In [12]:
def removePare(text):
    clean = []
    marked = []
    for i in text:
        i = i.strip()
        j = i.split('\t')
        if (len(j) == 1):
            continue
        elif ("（" == j[1][0]) \
            or ("@" in i and "）" in i[-1]) \
            or (("（" not in i) and ("）" in i[-1])):
            marked.append("pare:\t"+i)
            continue
        elif ("（" in i and "）" not in i):
            iSplit = i.split("（")
            j = "".join(iSplit[:-1])
            marked.append(j)
            marked.append("pare:\t（"+iSplit[-1])
            clean.append(j)
        else:
            marked.append(i)
            clean.append(i)
    clean = "\n".join(clean)
    return marked, clean

def stepFour(path, encoding):
    with open(path, encoding=encoding) as f:
        text = f.readlines()    
    clean = removePare(text)
    return clean

In [13]:
def extractPalette(textLst):
    names = []
    for i in textLst:
        names.append(i[0])
    names = list(set(names))
    palette = {}
    for key in names:
        palette[key] = (0x0, 0x0, 0x0)
    palette["pare"] = (241, 220, 167)
    return palette

def stepFive(text):
    textLst = text.split("\n")
    textLstSplit = [i.split(":\t") for i in textLst]
    print(extractPalette(textLstSplit))
    return textLst

In [14]:
def writeBleached(textLst, path, font, fontSize):
    document = Document()
    document.styles['Normal'].font.name = font  
    document.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), font)  
    document.styles['Normal'].font.size = Pt(fontSize) 
    paragraph = document.add_paragraph()
    textLstSplit = [i.split(":\t") for i in textLst]
    for i in textLstSplit:
        run = paragraph.add_run(':\t'.join(i)+'\n')
        rgb = palette[i[0]]
        run.font.color.rgb = RGBColor(rgb[0], rgb[1], rgb[2])
    document.save(path)
    
def writeRaw(textLst, path, font, fontSize):
    document = Document()
    document.styles['Normal'].font.name = font  
    document.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), font)  
    document.styles['Normal'].font.size = Pt(fontSize) 
    paragraph = document.add_paragraph()
    textLstSplit = [i.split(":\t") for i in textLst]
    for i in textLstSplit:
        if i[0] == "pare":
            run = paragraph.add_run(':\t'.join(i[1:])+'\n')
            rgb = palette[i[0]]
            run.font.color.rgb = RGBColor(rgb[0], rgb[1], rgb[2])
        else:
            run = paragraph.add_run(':\t'.join(i)+'\n')
            rgb = palette[i[0]]
            run.font.color.rgb = RGBColor(rgb[0], rgb[1], rgb[2])
    document.save(path)
    
def stepSix(textLstRaw, textLstBleached, projectPath, toPath, font, fontSize):
    pathRaw = projectPath + 'RAW-' + toPath.split('.')[0] + '.docx'
    writeRaw(textLstRaw, pathRaw, font, fontSize)
    pathBleached = projectPath + 'BLEACHED-' + toPath.split('.')[0] + '.docx'
    writeBleached(textLstBleached, pathBleached, font, fontSize)

In [15]:
#####################
#    START HERE     #
#####################

###########################
#    PHASE I: CLEANING    #
###########################

In [16]:
PROJECT_PATH = "log/狗咬胶/"
FROM_PATH = "《狗咬胶》原文件.txt"
TO_PATH = "狗咬胶.txt"

FROM_ENCODING = "gbk"
TO_ENCODING = "utf-8"

FONT = 'FangSong'
FONTSIZE = 10.5

# read file and draft alias
text = stepOne(PROJECT_PATH+FROM_PATH, FROM_ENCODING)

{'骰娘': '', '岸谷胜': '', '海音寺佑': '', '海音寺 佑': '', '佐竹摩衣': '', '津波时化': '', '信原真辉': ''}


In [17]:
# fill the correct alias
alias = {'骰娘': 'bot', '海音寺 佑': '海音寺佑', '津波时化': '寒河江春希'}


In [18]:
# clean by line
clean = stepTwo(text, alias)

# check result in jupyter notebook
print(clean)

海音寺佑:	（可以！）
佐竹摩衣:	（鼓掌
海音寺佑:	（那来了）
海音寺佑:	……在甜美的香味中睁开眼。回过神来，你发现自己正身处陌生的房间。小夜灯柔和的昏黄灯光对眼睛很友善，但总觉得有些暗了。
岸谷胜:	……这是哪儿？
岸谷胜:	也许又是一场噩梦，而举着火把的村民很快就要破门而入了。
海音寺佑:	想要看清周围环境的你稍微侧过头后，眼前是让你的后脑勺搁在自己膝盖上的海音寺佑。他只披着领口大敞的衬衫，注意到你的动作后微笑着这么说道。“啊，早安，小胜。开始今天的治疗吧。”
岸谷胜:	“你那穿的什么东西？”
岸谷胜:	嫌恶一样转开头去。
海音寺佑:	与此同时，能看见裸露出来的肩头有层层叠叠的咬痕。不知道已经被反复啃咬过多少次，那里形成了轻微的淤伤。看见那伤痕的瞬间，你脑海里有什么开始窃窃私语。像是引诱你一般的蛊惑笑声。——啊啊，好想咬下去、吞吃入腹由于突如其来的状况SC（0/1）以及POW*5
海音寺佑:	（开团即爬山）
岸谷胜:	无法抑制的渴望萌生出来了。
bot:	[岸谷胜]的理智检定结果: 26/90=成功你的理智值减少0=0点，当前剩余90点总之，世上总有些事是超乎常人所能理解的。
岸谷胜:	好歹稳住了心神……这想必是什么恶劣的玩笑。
海音寺佑:	（总之，世上总有些事是超乎常人所能理解的）
bot:	[岸谷胜]进行意志鉴定: D100=6/90 极难成功。失敬失敬，竟然形容先生是个奇人。
海音寺佑:	（…我笑喷了）
佐竹摩衣:	（爬多了x
寒河江春希:	（让老子咬就咬岂不是很没面子
岸谷胜:	“嘁。这算什么情况啊，海音寺？”
海音寺佑:	你总算是抑制住了那种冲动吧。见你这样，海音寺似乎有些不知所措，只好柔声劝你。“不行的啦。因为这是治疗……再讨厌的话，不咬下去的话也没法治好吧？”这么说着的他也没把衬衫穿回去，任由那带着伤口的肩膀展示在你面前。不过，看着怎样都不愿意下嘴的你，他露出似乎在思索着什么的表情。
岸谷胜:	“你不觉得很怪吗？明明怪的是你吧？”站起来离他有着体温的皮肤远一点，“这到底是哪门子治疗——这些难不成都是我咬的？”
岸谷胜:	“这是哪儿？你起码回答一个问题。”
海音寺佑:	“是，是的啊？”他呆呆地看着你。随后便低下头去。“…唉，我知道小胜肯定不喜欢这样……可是不那样做的话你会很痛苦啊，我也不想看到你那样。”海音寺有些纠结地捏着自己的手。
岸谷胜:	

In [19]:
# encoding and write file
stepThree(clean, TO_ENCODING, PROJECT_PATH+'RAW-'+TO_PATH)

In [20]:
###############################
#    PHASE II: REMOVE PARE    #
###############################

In [21]:
# remove parentheses and review
marked, clean = stepFour(PROJECT_PATH+'RAW-'+TO_PATH, TO_ENCODING)
print(clean)

海音寺佑:	……在甜美的香味中睁开眼。回过神来，你发现自己正身处陌生的房间。小夜灯柔和的昏黄灯光对眼睛很友善，但总觉得有些暗了。
岸谷胜:	……这是哪儿？
岸谷胜:	也许又是一场噩梦，而举着火把的村民很快就要破门而入了。
海音寺佑:	想要看清周围环境的你稍微侧过头后，眼前是让你的后脑勺搁在自己膝盖上的海音寺佑。他只披着领口大敞的衬衫，注意到你的动作后微笑着这么说道。“啊，早安，小胜。开始今天的治疗吧。”
岸谷胜:	“你那穿的什么东西？”
岸谷胜:	嫌恶一样转开头去。
海音寺佑:	与此同时，能看见裸露出来的肩头有层层叠叠的咬痕。不知道已经被反复啃咬过多少次，那里形成了轻微的淤伤。看见那伤痕的瞬间，你脑海里有什么开始窃窃私语。像是引诱你一般的蛊惑笑声。——啊啊，好想咬下去、吞吃入腹由于突如其来的状况SC（0/1）以及POW*5
岸谷胜:	无法抑制的渴望萌生出来了。
bot:	[岸谷胜]的理智检定结果: 26/90=成功你的理智值减少0=0点，当前剩余90点总之，世上总有些事是超乎常人所能理解的。
岸谷胜:	好歹稳住了心神……这想必是什么恶劣的玩笑。
bot:	[岸谷胜]进行意志鉴定: D100=6/90 极难成功。失敬失敬，竟然形容先生是个奇人。
岸谷胜:	“嘁。这算什么情况啊，海音寺？”
海音寺佑:	你总算是抑制住了那种冲动吧。见你这样，海音寺似乎有些不知所措，只好柔声劝你。“不行的啦。因为这是治疗……再讨厌的话，不咬下去的话也没法治好吧？”这么说着的他也没把衬衫穿回去，任由那带着伤口的肩膀展示在你面前。不过，看着怎样都不愿意下嘴的你，他露出似乎在思索着什么的表情。
岸谷胜:	“你不觉得很怪吗？明明怪的是你吧？”站起来离他有着体温的皮肤远一点，“这到底是哪门子治疗——这些难不成都是我咬的？”
岸谷胜:	“这是哪儿？你起码回答一个问题。”
海音寺佑:	“是，是的啊？”他呆呆地看着你。随后便低下头去。“…唉，我知道小胜肯定不喜欢这样……可是不那样做的话你会很痛苦啊，我也不想看到你那样。”海音寺有些纠结地捏着自己的手。
岸谷胜:	“比起这莫名其妙的咬来咬去肯定有什么办法吧。”
岸谷胜:	“之前怎么回事？”
海音寺佑:	“这是哪儿……我也说不上来，实在要形容的话，你就当作是暂住的病房吧。”他有些失落地看着你。
岸谷胜:	“哈？你这什么意思？”
岸谷胜:	环顾四周，这个房间有门

In [22]:
# encoding and write file
stepThree(clean, TO_ENCODING, PROJECT_PATH+'BLEACHED-'+TO_PATH)

In [23]:
###############################
#      PHASE III: DYEING      #
###############################

In [24]:
clean = stepFive(clean)

{'bot': (0, 0, 0), '海音寺佑': (0, 0, 0), '岸谷胜': (0, 0, 0), 'pare': (241, 220, 167)}


In [25]:
palette = {'bot': (246, 114, 128), '海音寺佑': (40, 82, 122), '岸谷胜': (145, 9, 30), 'pare': (248, 181, 149)}

In [26]:
stepSix(marked, clean, PROJECT_PATH, TO_PATH, FONT, FONTSIZE)